---
# Linear regression exercises

---
These are introductory exercises in Machine learning with focus in **linear regression** .

<p class = "alert alert-info" role="alert"><b>Note</b> all datasets used in this exercise can be found under Data folder of the course Github repo</p>

<p class = "alert alert-info" role="alert"><b>Note</b> that in cases when you start to repeat code, try not to. Create functions to reuse code instead. </p>

<p class = "alert alert-info" role="alert"><b>Remember</b> to use <b>descriptive variable, function, index </b> and <b> column names</b> in order to get readable code </p>

The number of stars (\*), (\*\*), (\*\*\*) denotes the difficulty level of the task

---

## 0. Simulate phone dataset (*)

We want to simulate data $(x,y)$ to represent cost for phone subscriptions, with: 

- $x$ - called minutes per month
- $y$ - SEK per month 

&nbsp; a) Use ```numpy.random.normal()``` to simulate a dataset with the following requirements:(*)
- set a seed to 42 (for reproducibility and reference)
- simulate 400 x-values from the r.v. $X \sim \mathcal{N}(100, 100)$ 
- take absolute value of these x-values
- simulate noise 400 noise values from r.v. $\epsilon \sim \mathcal{N(0, 50)}$ 
- Let $y = 2x+25+\epsilon$
- plot the data set 

&nbsp; b) Now we want to remove some outliers according to this assumption: (*)
- no one talks more than 300 min using this type of subscription
- no ones costs can be negative
- plot the new dataset
- also plot ground truth using the true parameters $\beta_0 = 25, \beta_1 = 2$

&nbsp; c) Insert the values into a DataFrame (*)

<details>

<summary>Answer</summary>

<img src="../assets/simulated_phone_dataset_0.png" height="200"/>

a) 

Number of points x ≥ 300 min: 8

Number of points y < 0 kr: 6


b)

Length of x, outliers removed 386

Length of y, outliers removed 386

c)

df.head()

|    |   Minutes |     Cost |
|---:|----------:|---------:|
|  0 |   59.4428 | 168.721  |
|  1 |   40.0625 |  98.2118 |
|  2 |  100.524  | 258.433  |
|  3 |  104.698  | 310.548  |
|  4 |   54.9935 | 123.279  |


</details>

---

In [26]:
import numpy as np
import pandas as pd
import matplotlib as plt


#plt.style.use("seaborn_white")


np.random.seed(42)

X =np.random.normal(loc=100, scale=100,size=400)
e = np.random.normal(loc=0,scale=50, size=400 )
y = 2*X + 25 + e


y

array([ 244.62144766,  167.37838862,  354.79989261,  531.95500097,
        155.66605148,  209.31510523,  487.46154163,  371.36797158,
        137.1199044 ,  359.23395042,  167.89720534,   75.62194469,
        196.68674578,  -93.77220784, -103.3678659 ,   75.11816732,
         99.99137471,  293.63319823,  102.3600441 ,  -54.0848162 ,
        621.16715003,  267.61178202,  226.05743351,  -11.37108969,
        148.39225257,  315.61609582,  -53.44488851,  334.44217667,
        157.79348636,   78.72427572,   45.49575192,  493.49402801,
        208.83021333,   49.334927  ,  464.62683503,  -15.46399097,
        348.20349628, -235.93909769, -125.80633175,  261.59486223,
        391.89658845,  257.63891883,   98.49823852,  160.32325891,
       -135.9278731 ,  114.51478576,  151.20215811,  389.43045593,
        268.03031205, -180.56870717,  286.68283902,  195.74065994,
         40.32929762,  372.53758354,  404.68702358,  371.61238221,
         51.80497736,  111.39540871,  263.57022101,  360.21513